## TODO:
- Experiment with base longformer embedding + FFNN head to output scores
- Longformer might not be necessary as adding prompt text overfits the small train data
  - -> we have an extrapolative task i.e. low train variance high test variance
- For the train set only, try training on competition metric, so backprop from the estimate mcrmse

### Basics
- Create dataset class for the train data to load + iteratively batch and tokenise samples
  - pass tokenizer class / name through constructor
  - dataloader custom collate function to add padding ? using tokenizer pad id
- Try domain fine-tuning on commonlit texts + supervised training on comp dataset

In [1]:
import torch
from sentence_transformers import SentenceTransformer, models

## 1 - Define and load a model

In [2]:
model_name = 'sentence-transformers/all-distilroberta-v1'

word_embedding_model = models.Transformer(model_name, max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/Users/chris/repos/student-summary-evaluation/.env/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


## 2 - Check model is trainable

In [15]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [16]:
print_trainable_parameters(model)

trainable params: 82118400 || all params: 82118400 || trainable%: 100.0


## 3 - Train on sample data

In [6]:
x = ['some random input sequence', 'another input sequence of a slightly different length']
tokens = model.tokenize(x)
output = model.forward(tokens)

In [10]:
# Sentence embeddings are found in this entry, so apply loss function and 
# back propogate
output['sentence_embedding'].shape

torch.Size([2, 768])

In [14]:
output_emb = output['sentence_embedding']

# Generate some random vector to create a dummy loss function
gt = torch.randn_like(output_emb) 
loss = ((output_emb - gt)**2).mean()

loss.backward()